# Calcul du plafond de la PAJE (pour une *année incomplète*)

Simulateur pajemploi: https://www.pajemploi.urssaf.fr/pajeweb/simulerMensualisation.htm

Règles de calcul: https://www.pajemploi.urssaf.fr/pajewebinfo/files/live/sites/pajewebinfo/files/contributed/pdf/employeur_ama/ExempleRemunerationAnneeIncompleteAMA.pdf

In [ ]:
import math

import ipywidgets
from ipywidgets import interact

In [ ]:
# Constantes ####

nb_sem_par_an = 52
plafond_journalier_euro_net = 38.12    # euro net / j

In [ ]:
# Variables #####

# `nb_sem_conges_payes` : nombre de semaines de congés payés i.e. nombre de semaines où l'enfant ne sera pas accueilli par le salarié pour congés payés et/ou congés sans solde
# `nb_sem_sans_garde` : nombre de semaines où l'enfant ne sera pas confié par les parents pour d'autres motifs (congés des parents, RTT, accueil par les grands-parents, etc.)
# `salaire_horaire_net` : le salaire horaire net
# `nb_h_travaille_par_sem` : nombre d'heures de travail hebdomadaires
# `nb_jrs_travaille_par_sem` : nombre de jours hebdomadaires moyen

nb_sem_conges_payes = 5        # sem
nb_jrs_travaille_par_sem = 3   # jrs / sem

In [ ]:
# Résultats #####

# `nb_sem_garde` : nombre de semaines de garde programmées = 52 - (`nb_sem_conges_payes` + `nb_sem_sans_garde`)
# `nb_hrs_mensualisees` : nombre d'heures normales mensualisées
# `nb_jrs_activite_mensuel` : nombre de jours d'activité mensualisés = `nb_sem_travail` * `nb_jrs_travaille_par_sem`
# `salaire_net_mensuel` : salaire net total mensualisé (hors indemnités d'entretien)

def calc(nb_sem_sans_garde=6, salaire_horaire_net=4., nb_hrs_travaille_par_sem=30, verbose=True):
    # Nombre de semaines de garde programmées
    nb_sem_travail = nb_sem_par_an - (nb_sem_conges_payes + nb_sem_sans_garde)
    if verbose:
        print("Nombre de semaines de garde programmées :", nb_sem_travail, "sem")
    
    # Nombre d'heures normales mensualisées
    nb_hrs_mensualisees = math.ceil(nb_hrs_travaille_par_sem * nb_sem_travail / 12.)
    if verbose:
        print("Nombre d'heures normales mensualisées :", nb_hrs_mensualisees, "hrs / mois")

    # Salaire net total mensualisé (hors indemnités d'entretien)
    salaire_net_mensuel = (salaire_horaire_net * nb_hrs_travaille_par_sem * nb_sem_travail) / 12.
    if verbose:
        print("Salaire net total mensualisé (hors indemnités d'entretien) :", salaire_net_mensuel, "euros net / mois")

    # Nombre de jours d'activité mensualisés
    nb_jrs_activite_mensuel = math.ceil(nb_jrs_travaille_par_sem * nb_sem_travail / 12.)
    if verbose:
        print("Nombre de jours d'activité mensualisés :", nb_jrs_activite_mensuel, "jrs / mois")

    # Salaire net quotidien
    salaire_net_quotidien = salaire_net_mensuel / nb_jrs_activite_mensuel
    if verbose:
        print("Salaire net quotidien :", salaire_net_quotidien, "euros net / j")

    # Dépacement du plafond de la PAJE
    depacement_euro_net_par_j = salaire_net_quotidien - plafond_journalier_euro_net
    if verbose:
        print()
        if depacement_euro_net_par_j > 0:
            print("DÉPACEMENT (dépacement de {:0.2f} euros net / j)".format(depacement_euro_net_par_j))
        else:
            print("OK ({:0.2f} euros net / j en dessous du plafond)".format(abs(depacement_euro_net_par_j)))

    return depacement_euro_net_par_j

In [ ]:
@interact(nb_sem_sans_garde=(0,16,1), salaire_horaire_net=(3.,5.,0.1), nb_hrs_travaille_par_sem=(20,40,1))
def widget_callback(nb_sem_sans_garde=6, salaire_horaire_net=4., nb_hrs_travaille_par_sem=30):
    depacement_euro_net_par_j = calc(nb_sem_sans_garde=nb_sem_sans_garde,
                                     salaire_horaire_net=salaire_horaire_net,
                                     nb_hrs_travaille_par_sem=nb_hrs_travaille_par_sem,
                                     verbose=True)